# Assignment: Segmenting and Clustering Neighborhoods in Toronto

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

1. <a href ="#item1">Download and explore data</a>

2. <a href="#item2">Explore the neighborhood in Toronto downtown</a>

3. <a href="#item3">Analyze Each Neighborhood</a>

4. <a href="#item4">Cluster Neighborhoods</a>
  
</font>
</div>

#### Import libraries

In [3]:
pip install bs4

     |████████████████████████████████| 122kB 9.0MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/a0/b0/b2/4f80b9456b87abedbc0bf2d52235414c3467d8889be38dd472
Successfully built bs4
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install lxml

     |████████████████████████████████| 5.5MB 6.2MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [5]:
#import requests to get data 
import requests


from bs4 import BeautifulSoup #To clean data
import pandas as pd # To manipulate data

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

<a id='item1'></a>

## 1.Download and explore data

In [6]:
page = requests.get('https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&direction=prev&oldid=926287641')
contents = page.content

In [7]:
soup = BeautifulSoup(contents, 'lxml')
table = soup.find_all('table')[0] 

#### Transform data into pandas dataframe

In [8]:
df = pd.read_html(str(table),encoding='utf-8', header=0)[0]
df

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
...,...,...,...
283,M8Z,Etobicoke,Mimico NW
284,M8Z,Etobicoke,The Queensway West
285,M8Z,Etobicoke,Royal York South West
286,M8Z,Etobicoke,South of Bloor


#### Remove the cells where borough that is Not assigned

In [12]:
df = df[df['Borough']!= 'Not assigned']
df

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
...,...,...,...
282,M8Z,Etobicoke,Kingsway Park South West
283,M8Z,Etobicoke,Mimico NW
284,M8Z,Etobicoke,The Queensway West
285,M8Z,Etobicoke,Royal York South West


#### Combine the neighborhoods sharing the same postal code

In [15]:
neighborhood = df.groupby(by = 'Postcode')['Neighbourhood'].apply(','.join).reset_index()
new_df = pd.merge(left = df[['Postcode','Borough']],right = neighborhood,how='left', left_on='Postcode', right_on='Postcode')

new_df.drop_duplicates(subset ="Postcode", keep = 'first', inplace = True) 
new_df

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
4,M6A,North York,"Lawrence Heights,Lawrence Manor"
6,M7A,Queen's Park,Not assigned
...,...,...,...
193,M8X,Etobicoke,"The Kingsway,Montgomery Road,Old Mill North"
196,M4Y,Downtown Toronto,Church and Wellesley
197,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern
198,M8Y,Etobicoke,"Humber Bay,King's Mill Park,Kingsway Park Sout..."


#### Assign the Borough names to the not assigned neighbourhoods

In [17]:
target_rows = new_df['Neighbourhood']== 'Not assigned'
new_df.loc[target_rows,'Neighbourhood'] = new_df.loc[target_rows,'Borough']
new_df.sort_index()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
4,M6A,North York,"Lawrence Heights,Lawrence Manor"
6,M7A,Queen's Park,Queen's Park
...,...,...,...
193,M8X,Etobicoke,"The Kingsway,Montgomery Road,Old Mill North"
196,M4Y,Downtown Toronto,Church and Wellesley
197,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern
198,M8Y,Etobicoke,"Humber Bay,King's Mill Park,Kingsway Park Sout..."


#### Indicate the shape of table

In [18]:
new_df.shape

(103, 3)

#### Adding the longitude and latitude to dataframe

Download data and transform into dataframe

In [19]:
!wget -q -O 'coordinate.csv' http://cocl.us/Geospatial_data

In [21]:
cordinates = pd.read_csv('coordinate.csv')
cordinates = cordinates.rename(columns = {'Postal Code':'Postcode'})
cordinates

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


Identify the lon and lat of each targeted area by postcode 

In [24]:
df_coor = pd.merge(left = new_df[['Postcode','Borough','Neighbourhood']],right = cordinates,how='left', left_on='Postcode', right_on='Postcode')
df_coor

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway,Montgomery Road,Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558
101,M8Y,Etobicoke,"Humber Bay,King's Mill Park,Kingsway Park Sout...",43.636258,-79.498509


<a id='item2'></a>

## 2. Explore the neighborhood in Toronto downtown

#### Use geopy library to get the latitude and longitude values of Toronto.

In [26]:
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.8.3
  latest version: 4.8.4

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    openssl-1.1.1g             |       h516909a_1         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.2 MB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-2.0.0

#### Create a map of Toronto with neighborhoods superimposed on top.

In [31]:
address = 'Toronto'

geolocator = Nominatim(user_agent="TOR_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [32]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neiborhd in zip(df_coor['Latitude'], df_coor['Longitude'], df_coor['Borough'], df_coor['Neighbourhood']):
    label = '{}, {}'.format(neiborhd, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### Narrow to Downtown Toronto and create the same map 

In [34]:
downtown_data = df_coor[df_coor['Borough'] == 'Downtown Toronto'].reset_index(drop=True)


In [35]:
address = 'Downtown Toronto,Toronto'

geolocator = Nominatim(user_agent="TOR_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

In [36]:
# create map of East Toronto using latitude and longitude values
map_newdowntown = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neiborhd in zip(downtown_data['Latitude'], downtown_data['Longitude'], downtown_data['Borough'], downtown_data['Neighbourhood']):
    label = '{}, {}'.format(neiborhd, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newdowntown)  
    
map_newdowntown

#### Define Foursquare Credentials and Version

In [37]:
CLIENT_ID = '52NUPLMDCEXFRU551WWRUW23FL4ADZQQLTNXVFRW0GI1JY1T' # your Foursquare ID
CLIENT_SECRET = 'H3AS0LWWEBFW11GZO3AWFSTUZR0FTIIZ2O4EZ20CQCFZSHVN' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

First, let's create the GET request URL. Name your URL **url**.

In [38]:
# type your answer here
LIMIT = 50 # limit of number of venues returned by Foursquare API

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [39]:
downtown_venues = getNearbyVenues(names=downtown_data['Neighbourhood'],
                                   latitudes=downtown_data['Latitude'],
                                   longitudes=downtown_data['Longitude']
                                  )

Harbourfront,Regent Park
Ryerson,Garden District
St. James Town
Berczy Park
Central Bay Street
Christie
Adelaide,King,Richmond
Harbourfront East,Toronto Islands,Union Station
Design Exchange,Toronto Dominion Centre
Commerce Court,Victoria Hotel
Harbord,University of Toronto
Chinatown,Grange Park,Kensington Market
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown,St. James Town
First Canadian Place,Underground city
Church and Wellesley


In [40]:
downtown_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide,King,Richmond",50,50,50,50,50,50
Berczy Park,50,50,50,50,50,50
"CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara",16,16,16,16,16,16
"Cabbagetown,St. James Town",45,45,45,45,45,45
Central Bay Street,50,50,50,50,50,50
"Chinatown,Grange Park,Kensington Market",50,50,50,50,50,50
Christie,17,17,17,17,17,17
Church and Wellesley,50,50,50,50,50,50
"Commerce Court,Victoria Hotel",50,50,50,50,50,50


<a id='item3'></a>

##  3. Analyze Each Neighborhood

In [42]:
# one hot encoding
downtown_onehot = pd.get_dummies(downtown_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
downtown_onehot['Neighborhood'] =downtown_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [downtown_onehot.columns[-1]] + list(downtown_onehot.columns[:-1])
manhattan_onehot = downtown_onehot[fixed_columns]

downtown_onehot.head()

,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,...,Thai Restaurant,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [43]:
downtown_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
downtown_grouped

,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar
0,"Adelaide,King,Richmond",0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.04,0.000000,...,0.00,0.020000,0.000000,0.00,0.00,0.00,0.02,0.000000,0.00,0.00
1,Berczy Park,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.00,0.000000,...,0.00,0.020000,0.000000,0.00,0.00,0.00,0.02,0.000000,0.00,0.00
2,"CN Tower,Bathurst Quay,Island airport,Harbourf...",0.000000,0.0625,0.0625,0.0625,0.125,0.1875,0.125,0.00,0.000000,...,0.00,0.000000,0.000000,0.00,0.00,0.00,0.00,0.000000,0.00,0.00
3,"Cabbagetown,St. James Town",0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.00,0.000000,...,0.00,0.022222,0.000000,0.00,0.00,0.00,0.00,0.000000,0.00,0.00
4,Central Bay Street,0.020000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.00,0.000000,...,0.00,0.000000,0.000000,0.00,0.00,0.00,0.02,0.000000,0.00,0.02
5,"Chinatown,Grange Park,Kensington Market",0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.00,0.000000,...,0.02,0.000000,0.000000,0.00,0.00,0.00,0.06,0.000000,0.04,0.02
6,Christie,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.00,0.000000,...,0.00,0.000000,0.000000,0.00,0.00,0.00,0.00,0.000000,0.00,0.00
7,Church and Wellesley,0.040000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.02,0.000000,...,0.00,0.020000,0.020000,0.02,0.00,0.00,0.00,0.000000,0.00,0.00
8,"Commerce Court,Victoria Hotel",0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.04,0.000000,...,0.00,0.000000,0.000000,0.00,0.00,0.00,0.02,0.000000,0.00,0.00
9,"Design Exchange,Toronto Dominion Centre",0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.02,0.000000,...,0.00,0.000000,0.000000,0.00,0.00,0.02,0.02,0.000000,0.00,0.00


In [44]:
num_top_venues = 5

for hood in downtown_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = downtown_grouped[downtown_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
                 venue  freq
0                 Café  0.08
1          Coffee Shop  0.08
2           Steakhouse  0.06
3           Restaurant  0.04
4  American Restaurant  0.04


----Berczy Park----
                venue  freq
0         Coffee Shop  0.08
1        Cocktail Bar  0.04
2  Seafood Restaurant  0.04
3              Bakery  0.04
4          Restaurant  0.04


----CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara----
              venue  freq
0   Airport Service  0.19
1    Airport Lounge  0.12
2  Airport Terminal  0.12
3       Coffee Shop  0.06
4           Airport  0.06


----Cabbagetown,St. James Town----
         venue  freq
0  Coffee Shop  0.07
1  Pizza Place  0.04
2          Pub  0.04
3       Bakery  0.04
4    Pet Store  0.04


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.16
1      Sandwich Place  0.06
2     Bubble Tea Shop  0.04
3  Italian Restaurant  0.04
4     

Write a function to sort the venues in descending order

In [45]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### Sort the venues in descending order

In [46]:
import numpy as np
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = downtown_grouped['Neighborhood']

for ind in np.arange(downtown_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(downtown_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Café,Coffee Shop,Steakhouse,Restaurant,Hotel,American Restaurant,Concert Hall,Noodle House,Mediterranean Restaurant,Plaza
1,Berczy Park,Coffee Shop,Seafood Restaurant,Cocktail Bar,Beer Bar,Restaurant,Cheese Shop,Farmers Market,Café,Bakery,Italian Restaurant
2,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Airport Service,Airport Lounge,Airport Terminal,Sculpture Garden,Harbor / Marina,Rental Car Location,Boat or Ferry,Bar,Coffee Shop,Airport
3,"Cabbagetown,St. James Town",Coffee Shop,Restaurant,Market,Pub,Pizza Place,Bakery,Pet Store,Italian Restaurant,Café,Plaza
4,Central Bay Street,Coffee Shop,Sandwich Place,Burger Joint,Café,Italian Restaurant,Bubble Tea Shop,Park,Poke Place,Ice Cream Shop,Office


<a id='item4'></a>

## 4. Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 10 clusters.

In [47]:
# set number of clusters
kclusters = 10

downtown_grouped_clustering = downtown_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(downtown_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([4, 0, 3, 9, 2, 8, 6, 2, 4, 4], dtype=int32)

In [48]:
downtown_data = downtown_data.rename(columns = {'Neighbourhood':'Neighborhood'})

In [49]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

downtown_merged = downtown_data
print(downtown_merged)
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
downtown_merged = downtown_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

downtown_merged.head() # check the last columns!


   Postcode           Borough  \
0       M5A  Downtown Toronto   
1       M5B  Downtown Toronto   
2       M5C  Downtown Toronto   
3       M5E  Downtown Toronto   
4       M5G  Downtown Toronto   
5       M6G  Downtown Toronto   
6       M5H  Downtown Toronto   
7       M5J  Downtown Toronto   
8       M5K  Downtown Toronto   
9       M5L  Downtown Toronto   
10      M5S  Downtown Toronto   
11      M5T  Downtown Toronto   
12      M5V  Downtown Toronto   
13      M4W  Downtown Toronto   
14      M5W  Downtown Toronto   
15      M4X  Downtown Toronto   
16      M5X  Downtown Toronto   
17      M4Y  Downtown Toronto   

                                         Neighborhood   Latitude  Longitude  
0                            Harbourfront,Regent Park  43.654260 -79.360636  
1                             Ryerson,Garden District  43.657162 -79.378937  
2                                      St. James Town  43.651494 -79.375418  
3                                         Berczy Park  43.64

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636,7,Coffee Shop,Park,Bakery,Pub,Café,Breakfast Spot,Theater,Dessert Shop,Performing Arts Venue,Chocolate Shop
1,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937,2,Café,Coffee Shop,Italian Restaurant,Clothing Store,Cosmetics Shop,Ramen Restaurant,Theater,Bookstore,Burrito Place,Comic Shop
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Café,Coffee Shop,Cosmetics Shop,Creperie,Hotel,Restaurant,Gastropub,Park,Seafood Restaurant,Farmers Market
3,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0,Coffee Shop,Seafood Restaurant,Cocktail Bar,Beer Bar,Restaurant,Cheese Shop,Farmers Market,Café,Bakery,Italian Restaurant
4,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,2,Coffee Shop,Sandwich Place,Burger Joint,Café,Italian Restaurant,Bubble Tea Shop,Park,Poke Place,Ice Cream Shop,Office


#### Visualize the resulting clusters

In [50]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(downtown_merged['Latitude'], downtown_merged['Longitude'], downtown_merged['Neighborhood'], downtown_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Examing the cluster

In [52]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 0, downtown_merged.columns[[1] + list(range(3, downtown_merged.shape[1]))]]

,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,43.651494,-79.375418,0,Café,Coffee Shop,Cosmetics Shop,Creperie,Hotel,Restaurant,Gastropub,Park,Seafood Restaurant,Farmers Market
3,Downtown Toronto,43.644771,-79.373306,0,Coffee Shop,Seafood Restaurant,Cocktail Bar,Beer Bar,Restaurant,Cheese Shop,Farmers Market,Café,Bakery,Italian Restaurant
14,Downtown Toronto,43.646435,-79.374846,0,Café,Coffee Shop,Restaurant,Cocktail Bar,Cheese Shop,Beer Bar,Italian Restaurant,Farmers Market,Seafood Restaurant,Japanese Restaurant
